In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as preproc
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder

In [4]:
data = pd.read_excel('Case1_data.xlsx')
data=data.drop(columns=['FlightNumber'])
is_NaN = data.isnull();row_has_NaN = is_NaN.any(axis=1);rows_with_NaN = data[row_has_NaN]
data = data.drop(list(rows_with_NaN.index))
data=data.reset_index()
data.AircraftType=data.AircraftType.astype('str')

def apply_OneHotEncoder(data_col):
    # creating instance of one-hot-encoder
    enc = OneHotEncoder(handle_unknown='ignore')
    # passing bridge-types-cat column (label encoded values of bridge_types)
    enc_df = pd.DataFrame(enc.fit_transform(data_col).toarray())
    d={}
    for i in range (0,len(np.unique(data_col))):
        d[i]=enc.categories_[0][i]
    enc_df=enc_df.rename(columns=d)
    return enc_df

DateTime_dic = {'Month':[],'Year':[],'Hour':[],'Minute':[],'Day_Of_Year':[],'Day_Of_Week':[],
               'Is_month_end':[],'Is_month_start':[],'Quarter_end':[],'Quarter_start':[],'Year_end':[],
               'Year_Start':[],'Quarter':[]}

for a in data['ScheduleTime']:
    DateTime_dic['Month'].append(a.month);DateTime_dic['Year'].append(a.year)
    DateTime_dic['Hour'].append(a.hour);DateTime_dic['Minute'].append(a.minute);
    DateTime_dic['Day_Of_Year'].append(a.dayofyear);DateTime_dic['Day_Of_Week'].append(a.dayofweek);
    DateTime_dic['Is_month_end'].append(int(a.is_month_end));
    DateTime_dic['Is_month_start'].append(int(a.is_month_start));
    DateTime_dic['Quarter_end'].append(int(a.is_quarter_end));
    DateTime_dic['Quarter_start'].append(int(a.is_quarter_start));
    DateTime_dic['Year_end'].append(int(a.is_year_end));DateTime_dic['Year_Start'].append(int(a.is_year_start));
    DateTime_dic['Quarter'].append(a.quarter)
    
pd_time=pd.DataFrame.from_dict(DateTime_dic)
y=np.array(data.LoadFactor)
data=data.drop(columns='LoadFactor')

df = pd.concat([apply_OneHotEncoder(data[['Airline']]),apply_OneHotEncoder(data[['Destination']]),
                apply_OneHotEncoder(data[['AircraftType']]),apply_OneHotEncoder(data[['FlightType']]),
                apply_OneHotEncoder(data[['Sector']]),pd_time,pd.DataFrame(data.SeatCapacity)],axis=1)

In [19]:
df

AY   BJ   BT   BZ   CL   CN   DO   DT   DU   EM  ...  Day_Of_Year  \
0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...            1   
1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...            1   
2      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...            1   
3      0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...            1   
4      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...            1   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...          ...   
36763  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...           39   
36764  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...           39   
36765  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...           39   
36766  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...           39   
36767  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...           39   

       Day_Of_Week  Is_month_end  Is_month_start  Quarter_end  Quarter_start  \
0                4             0               1            0              1   
1                4             0               1            0              1   
2                4             0               1            0              1   
3                4             0               1            0              1   
4                4             0               1            0              1   
...            ...           ...             ...          ...            ...   
36763            1             0               0            0              0   
36764            1             0               0            0              0   
36765            1             0               0            0              0   
36766            1             0               0            0              0   
36767            1             0               0            0              0   

       Year_end  Year_Start  Quarter  SeatCapacity  
0             0           1        1           142  
1             0           1        1            74  
2             0           1        1           142  
3             0           1        1            72  
4             0           1        1           186  
...         ...         ...      ...           ...  
36763         0           0        1           180  
36764         0           0        1           156  
36765         0           0        1           156  
36766         0           0        1           189  
36767         0           0        1           189  

[36768 rows x 410 columns]

In [20]:
df.to_csv('Feature_selection_Lluis.csv')

In [6]:
# NORMALIZATION
X=np.array(df)
X_pre = X - np.mean(X,axis=0)
normalizer = preproc.Normalizer().fit(X_pre.T)  
X_pre = normalizer.transform(X_pre.T).T
min_max_scaler = preproc.MinMaxScaler()
y_pre = min_max_scaler.fit_transform(y.reshape(-1,1)).reshape(-1,)

In [11]:
X_pre_df=pd.DataFrame(X_pre)

In [12]:
y_pre_df=pd.DataFrame(y_pre)

In [17]:
norm_df=pd.concat([X_pre_df,y_pre_df],axis=1)
norm_df

0         1         2         3         4         5         6    \
0     -0.000516 -0.000353 -0.000329 -0.000207 -0.001874 -0.001384 -0.003146   
1     -0.000516 -0.000353 -0.000329 -0.000207 -0.001874 -0.001384 -0.003146   
2     -0.000516 -0.000353 -0.000329 -0.000207 -0.001874 -0.001384 -0.003146   
3     -0.000516 -0.000353 -0.000329 -0.000207 -0.001874  0.019656 -0.003146   
4     -0.000516 -0.000353 -0.000329 -0.000207 -0.001874 -0.001384 -0.003146   
...         ...       ...       ...       ...       ...       ...       ...   
36763 -0.000516 -0.000353 -0.000329 -0.000207 -0.001874 -0.001384 -0.003146   
36764 -0.000516 -0.000353 -0.000329 -0.000207 -0.001874 -0.001384 -0.003146   
36765 -0.000516 -0.000353 -0.000329 -0.000207 -0.001874 -0.001384 -0.003146   
36766 -0.000516 -0.000353 -0.000329 -0.000207  0.014513 -0.001384 -0.003146   
36767 -0.000516 -0.000353 -0.000329 -0.000207  0.014513 -0.001384 -0.003146   

            7         8         9    ...       401       402       403  \
0     -0.000027 -0.000047 -0.000424  ...  0.002488 -0.000945  0.028187   
1     -0.000027 -0.000047 -0.000424  ...  0.002488 -0.000945  0.028187   
2     -0.000027 -0.000047 -0.000424  ...  0.002488 -0.000945  0.028187   
3     -0.000027 -0.000047 -0.000424  ...  0.002488 -0.000945  0.028187   
4     -0.000027 -0.000047 -0.000424  ...  0.002488 -0.000945  0.028187   
...         ...       ...       ...  ...       ...       ...       ...   
36763 -0.000027 -0.000047 -0.000424  ... -0.005178 -0.000945 -0.000965   
36764 -0.000027 -0.000047 -0.000424  ... -0.005178 -0.000945 -0.000965   
36765 -0.000027 -0.000047 -0.000424  ... -0.005178 -0.000945 -0.000965   
36766 -0.000027 -0.000047 -0.000424  ... -0.005178 -0.000945 -0.000965   
36767 -0.000027 -0.000047 -0.000424  ... -0.005178 -0.000945 -0.000965   

            404       405       406       407       408       409       0    
0     -0.000531  0.048399 -0.000274  0.091910 -0.008896 -0.001177  0.276882  
1     -0.000531  0.048399 -0.000274  0.091910 -0.008896 -0.007203  0.128248  
2     -0.000531  0.048399 -0.000274  0.091910 -0.008896 -0.001177  0.386681  
3     -0.000531  0.048399 -0.000274  0.091910 -0.008896 -0.007380  0.225962  
4     -0.000531  0.048399 -0.000274  0.091910 -0.008896  0.002722  0.138493  
...         ...       ...       ...       ...       ...       ...       ...  
36763 -0.000531 -0.000562 -0.000274 -0.000296 -0.008896  0.002190  0.354006  
36764 -0.000531 -0.000562 -0.000274 -0.000296 -0.008896  0.000063  0.360669  
36765 -0.000531 -0.000562 -0.000274 -0.000296 -0.008896  0.000063  0.408469  
36766 -0.000531 -0.000562 -0.000274 -0.000296 -0.008896  0.002988  0.283349  
36767 -0.000531 -0.000562 -0.000274 -0.000296 -0.008896  0.002988  0.337149  

[36768 rows x 411 columns]

In [18]:
norm_df.to_csv('Normalized_Lluis.csv') 